## Exercise 1: Intersectional Fairness

Using the COMPAS dataset to investigate the impact of protected attributes like race and sex with and without age constraints.


In [2]:
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_compas
from aif360.algorithms.preprocessing import Reweighing
from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric
from sklearn.linear_model import LogisticRegression
import numpy as np

compas_data = load_preproc_data_compas()

### Case 1: Race as protected attribute

We fix the bias using the reweighing preprocessing technique and then measure the bias assuming sex is the protected attribute.

First let's download the dataset and partition it into training and testing datasets:

In [3]:
compas_data = load_preproc_data_compas()
dataset_orig_train_race, dataset_orig_test_race = compas_data.split([0.7], shuffle=True, seed=0)

# Training with the base

In [6]:
dataset_orig_train_base, dataset_orig_test_base = compas_data.split([0.7], shuffle=True, seed=0)

clf_base = LogisticRegression(solver='lbfgs', max_iter=1000, C=1.0, penalty='l2', random_state=0)
clf_base.fit(dataset_orig_train_base.features, dataset_orig_train_base.labels.flatten())
dataset_bias_test_pred_base = clf_base.predict(dataset_orig_test_base.features)

dataset_bias_test_base = dataset_orig_test_base.copy()
dataset_bias_test_base.scores = dataset_bias_test_pred_base
dataset_bias_test_base.labels = dataset_orig_test_base.labels

test_df_base = dataset_bias_test_base.convert_to_dataframe()[0]
test_df_base['model_not_recid'] = dataset_bias_test_base.scores.flatten()
test_df_base['observed_not_recid'] = 1 - test_df_base['two_year_recid']

dataset_base = StandardDataset(test_df_base, label_name='model_not_recid', favorable_classes=[0],
                 protected_attribute_names=['sex'],
                 privileged_classes=[[1]],
                 instance_weights_name=None)


metric_test_sex = BinaryLabelDatasetMetric(dataset_base,
                                           unprivileged_groups=[{'sex': 0}],
                                           privileged_groups=[{'sex': 1}])
print("Test set: Difference in mean outcomes between unprivileged and privileged groups (sex) = %f" % metric_test_sex.mean_difference())

Test set: Difference in mean outcomes between unprivileged and privileged groups (sex) = -0.228834


In [8]:
dataset_orig_train_base, dataset_orig_test_base = compas_data.split([0.7], shuffle=True, seed=0)

clf_base = LogisticRegression(solver='lbfgs', max_iter=1000, C=1.0, penalty='l2', random_state=0)
clf_base.fit(dataset_orig_train_base.features, dataset_orig_train_base.labels.flatten())
dataset_bias_test_pred_base = clf_base.predict(dataset_orig_test_base.features)

dataset_bias_test_base = dataset_orig_test_base.copy()
dataset_bias_test_base.scores = dataset_bias_test_pred_base
dataset_bias_test_base.labels = dataset_orig_test_base.labels

test_df_base = dataset_bias_test_base.convert_to_dataframe()[0]
test_df_base['model_not_recid'] = dataset_bias_test_base.scores.flatten()
test_df_base['observed_not_recid'] = 1 - test_df_base['two_year_recid']

dataset_base = StandardDataset(test_df_base, label_name='model_not_recid', favorable_classes=[0],
                 protected_attribute_names=['race'],
                 privileged_classes=[[1]],
                 instance_weights_name=None)


metric_test_sex = BinaryLabelDatasetMetric(dataset_base,
                                           unprivileged_groups=[{'race': 0}],
                                           privileged_groups=[{'race': 1}])
print("Test set: Difference in mean outcomes between unprivileged and privileged groups (sex) = %f" % metric_test_sex.mean_difference())

Test set: Difference in mean outcomes between unprivileged and privileged groups (sex) = -0.276763


In [9]:
test_df_base

,sex,race,age_cat=25 to 45,age_cat=Greater than 45,age_cat=Less than 25,priors_count=0,priors_count=1 to 3,priors_count=More than 3,c_charge_degree=F,c_charge_degree=M,two_year_recid,model_not_recid,observed_not_recid
5219,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
7435,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1099,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
3648,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0
208,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10293,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
6796,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
3493,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
5472,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


We now set 'race' as the protected attribute and perform the RW technique on the train dataset. It's important to remove sex from the protected attributes, because the default protected attributes for the dataset are both 'race' and 'sex'.

In [14]:
privileged_groups_race = [{'race': 1}]
unprivileged_groups_race = [{'race': 0}]

rw_race = Reweighing(unprivileged_groups=unprivileged_groups_race, privileged_groups=privileged_groups_race)
dataset_transf_train_race = rw_race.fit_transform(dataset_orig_train_race)

dataset_transf_train_race.protected_attribute_names.remove('sex')
print(dataset_transf_train_race.protected_attribute_names)

['race']


To avoid unnecessary code we don't perform the bias validation, but at this point the instance weights are modified to correct the bias considering only 'race'. Let's now train a logistic regression on the reweighed version of the dataset.

Additionally, let's use the model to perform predictions on the test set.

In [15]:
clf_race = LogisticRegression(solver='lbfgs', max_iter=1000, C=1.0, penalty='l2', random_state=0)
clf_race.fit(dataset_transf_train_race.features, dataset_transf_train_race.labels.flatten())

dataset_bias_test_pred_race = clf_race.predict(dataset_orig_test_race.features)

To put this prediction scores into context let's create a dataframe containing both the original label and the predicted scores using the model:

In [16]:
dataset_bias_test_sex = dataset_orig_test_race.copy()
dataset_bias_test_sex.scores = dataset_bias_test_pred_race
dataset_bias_test_sex.labels = dataset_orig_test_race.labels

test_df_sex = dataset_bias_test_sex.convert_to_dataframe()[0]
test_df_sex['model_not_recid'] = dataset_bias_test_sex.scores.flatten()
test_df_sex['observed_not_recid'] = 1 - test_df_sex['two_year_recid']

Finally, in order to measure the bias but considering the model's prediction and 'sex' as the protected attribute, we need to create a standard dataset based on the dataframe. Note that the label is set to the model's prediction and not the original label value.

In [17]:
dataset_sex = StandardDataset(test_df_sex, label_name='model_not_recid', favorable_classes=[0],
                 protected_attribute_names=['sex'],
                 privileged_classes=[[1]],
                 instance_weights_name=None)

metric_test_sex = BinaryLabelDatasetMetric(dataset_sex,
                                           unprivileged_groups=[{'sex': 0}],
                                           privileged_groups=[{'sex': 1}])
print("Test set: Difference in mean outcomes between unprivileged and privileged groups (sex) = %f" % metric_test_sex.mean_difference())

Test set: Difference in mean outcomes between unprivileged and privileged groups (sex) = -0.228834


We can see clearly here that the model trained on the RW process performed for the 'race' attribute changes a lot the outcomes considering 'sex'. The model creates a bigger bias for the attribute that wasn't considered.

### Case 2: Sex as protected attribute

Let's now follow the same logic but focusing first on 'sex' as the protected attribute and then analyzing the bias on 'race':

In [142]:
compas_data = load_preproc_data_compas()
dataset_orig_train_sex, dataset_orig_test_sex = compas_data.split([0.7], shuffle=True, seed=0)

privileged_groups_sex = [{'sex': 1}]
unprivileged_groups_sex = [{'sex': 0}]

rw_sex = Reweighing(unprivileged_groups=unprivileged_groups_sex, privileged_groups=privileged_groups_sex)
dataset_transf_train_sex = rw_sex.fit_transform(dataset_orig_train_sex)

dataset_transf_train_sex.protected_attribute_names.remove('race')
print(dataset_transf_train_sex.protected_attribute_names)

clf_sex = LogisticRegression(solver='lbfgs', max_iter=1000, C=1.0, penalty='l2', random_state=0)
clf_sex.fit(dataset_transf_train_sex.features, dataset_transf_train_sex.labels.flatten())

dataset_bias_test_pred_sex = clf_sex.predict(dataset_orig_test_sex.features)

dataset_bias_test_race = dataset_orig_test_sex.copy()
dataset_bias_test_race.scores = dataset_bias_test_pred_sex
dataset_bias_test_race.labels = dataset_orig_test_sex.labels

test_df_race = dataset_bias_test_race.convert_to_dataframe()[0]
test_df_race['model_not_recid'] = dataset_bias_test_race.scores.flatten()
test_df_race['observed_not_recid'] = 1 - test_df_race['two_year_recid']

dataset_race = StandardDataset(test_df_race, label_name='model_not_recid', favorable_classes=[0],
                 protected_attribute_names=['race'],
                 privileged_classes=[[1]],
                 instance_weights_name=None)

metric_test_race = BinaryLabelDatasetMetric(dataset_race,
                                           unprivileged_groups=[{'race': 0}],
                                           privileged_groups=[{'race': 1}])
print("Test set: Difference in mean outcomes between unprivileged and privileged groups (race) = %f" % metric_test_race.mean_difference())

['sex']
Test set: Difference in mean outcomes between unprivileged and privileged groups (race) = -0.276763


This other model has a different impact, which is normal considered that the two models are trained on different conditions. However the same analysis can be made of this result: the model creates a bigger bias because it disregards 'race' when being built.

### Case 3: Age parameter

We perform the same analysis with a truncated version of the dataset, including only entries with people of age < 25

First considering the initial case, training on 'race' and measuring on 'sex'

In [143]:
test_df_sex_age = test_df_sex[test_df_sex['age_cat=Less than 25'] == 1]

dataset_sex_age = StandardDataset(test_df_sex_age, label_name='model_not_recid', favorable_classes=[0],
                 protected_attribute_names=['sex'],
                 privileged_classes=[[1]],
                 instance_weights_name=None)

metric_test_sex_age = BinaryLabelDatasetMetric(dataset_sex_age,
                                           unprivileged_groups=[{'sex': 0}],
                                           privileged_groups=[{'sex': 1}])
print("Test set: Difference in mean outcomes between unprivileged and privileged groups (sex) = %f" % metric_test_sex_age.mean_difference())

Test set: Difference in mean outcomes between unprivileged and privileged groups (sex) = -0.421062


We can see how the effect of this model intensifies and the bias grows even bigger when considering only groups of records with age <25. It's important to understand additionally that there are less records, and therefore bias impacts are multiplied.

Now let's see if we have the same behavior with the other case:

In [144]:
test_df_race_age = test_df_race[test_df_race['age_cat=Less than 25'] == 1]

dataset_race_age = StandardDataset(test_df_race_age, label_name='model_not_recid', favorable_classes=[0],
                 protected_attribute_names=['race'],
                 privileged_classes=[[1]],
                 instance_weights_name=None)

metric_test_race_age = BinaryLabelDatasetMetric(dataset_race_age,
                                           unprivileged_groups=[{'race': 0}],
                                           privileged_groups=[{'race': 1}])
print("Test set: Difference in mean outcomes between unprivileged and privileged groups (race) = %f" % metric_test_race_age.mean_difference())

Test set: Difference in mean outcomes between unprivileged and privileged groups (race) = -0.283674


In this second case the impact is not as big. This means that, regardless of the age, the predictions made by a model trained on 'sex' are similar in terms of 'race'.